In [8]:
import os

from llama_index.core import QueryBundle
from llama_index.llms.openai import OpenAI as OpenAILlama
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

In [6]:
openai_key = os.environ.get("OPENAI_API_KEY")

reranker = RankGPTRerank(
    llm=OpenAILlama(
        model="gpt-3.5-turbo-16k",
        temperature=0.0,
        api_key=openai_key,
    ),
    top_n=5,
    verbose=True,
)

In [9]:
query = "query"
query_bundle = QueryBundle(query)

In [10]:
from llama_index.core.schema import Document, NodeWithScore
from llama_index.core.node_parser import SimpleNodeParser


def get_nodes_ddgs(documents):
    transformed_documents = []
    for doc in documents:
        node_metadata = {"title": doc["title"], "link": doc["url"]}
        document = Document()
        document.text = doc["text"]
        document.metadata = node_metadata
        transformed_documents.append(document)
    base_node_parser = SimpleNodeParser()
    nodes = base_node_parser.get_nodes_from_documents(transformed_documents)
    return [NodeWithScore(node=node, score=1.0) for node in nodes]

In [12]:
documents = [
    {
        "title": "Title 1",
        "url": "https://example.com/1",
        "text": "Text 1...",
    },
    {
        "title": "Title 2",
        "url": "https://example.com/2",
        "text": "Text 2...",
    },
    {
        "title": "Title 3",
        "url": "https://example.com/3",
        "text": "Text 3...",
    },
]

In [13]:
retrieved_nodes = get_nodes_ddgs(documents)

reranked_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)

After Reranking, new rank list for nodes: [0, 1, 2]

In [14]:
reranked_nodes

[NodeWithScore(node=TextNode(id_='fe829497-80bf-489b-bfad-c818279b498d', embedding=None, metadata={'title': 'Title 1', 'link': 'https://example.com/1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='9d2fee24-cf2c-473d-acc1-c8ffbadefb9c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': 'Title 1', 'link': 'https://example.com/1'}, hash='d19b076ea75d4657024a65a8dcac8a0b0b3eb6d2fab30980d79c1eaf06f18e29'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c595028e-1d38-4ac8-a9cb-a865c22fdec7', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d6bcd16ebaee8bc87b8ae935075678995e4d57472f8b4b6f16f694e462a22d4f')}, text='Text 1...', start_char_idx=0, end_char_idx=9, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1.0),
 NodeWithScore(node=TextNode(id_='c595028e-1d38-4ac8-a9cb-a865c22fdec7', embedding=None, metadata={'title': '